<a href="https://colab.research.google.com/github/busrasari/INF506-DataAnalysisMethods/blob/main/UE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1

In [ ]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")

def fetch_spam_data(ham_url=HAM_URL, spam_url=SPAM_URL, spam_path=SPAM_PATH):
   if not os.path.isdir(spam_path):
       os.makedirs(spam_path)
   for filename, url in (("ham.tar.bz2", ham_url), ("spam.tar.bz2", spam_url)):
       path = os.path.join(spam_path, filename)
       if not os.path.isfile(path):
           urllib.request.urlretrieve(url, path)
       tar_bz2_file = tarfile.open(path)
       tar_bz2_file.extractall(path=spam_path)
       tar_bz2_file.close()

fetch_spam_data()

HAM_DIR = os.path.join(SPAM_PATH, "easy_ham")
SPAM_DIR = os.path.join(SPAM_PATH, "spam")
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

print(len(ham_filenames))
print(len(spam_filenames))


2500
500


#2

In [ ]:
import email
import email.policy

def load_email(is_spam, filename, spam_path=SPAM_PATH):
   directory = "spam" if is_spam else "easy_ham"
   with open(os.path.join(spam_path, directory, filename), "rb") as f:
       return email.parser.BytesParser(policy=email.policy.default).parse(f)

ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

print(ham_emails[1].get_content().strip())


Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [ ]:
print(spam_emails[6].get_content().strip())

Help wanted.  We are a 14 year old fortune 500 company, that is
growing at a tremendous rate.  We are looking for individuals who
want to work from home.

This is an opportunity to make an excellent income.  No experience
is required.  We will train you.

So if you are looking to be employed from home with a career that has
vast opportunities, then go:

http://www.basetel.com/wealthnow

We are looking for energetic and self motivated people.  If that is you
than click on the link and fill out the form, and one of our
employement specialist will contact you.

To be removed from our link simple go to:

http://www.basetel.com/remove.html


4139vOLW7-758DoDY1425FRhM1-764SMFc8513fCsLl40


In [ ]:
print(spam_emails[0]["Subject"])


Life Insurance - Why Pay More?


#3

In [ ]:
import numpy as np
import re
from html import unescape

X = np.array(spam_emails + ham_emails, dtype=object)


def html_to_plain_text(html):
   text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
   text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
   text = re.sub('<.*?>', '', text, flags=re.M | re.S)
   text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
   return unescape(text)

def email_to_text(email):
   html = None
   for part in email.walk():
       ctype = part.get_content_type()
       if not ctype in ("text/plain", "text/html"):
           continue
       try:
           content = part.get_content()
       except: # in case of encoding issues
           content = str(part.get_payload())
       if ctype == "text/plain":
           return content
       else:
           html = content
   if html:
       return html_to_plain_text(html)


sample_html_spam = X[0]
print(html_to_plain_text(sample_html_spam.get_content()))



Save up to 70% on Life Insurance.
Why Spend More Than You Have To?
Life Quote Savings
    Ensuring your
      family's financial security is very important. Life Quote Savings makes
      buying life insurance simple and affordable. We Provide FREE Access to The
      Very Best Companies and The Lowest Rates.
          Life Quote Savings is FAST, EASY and
            SAVES you money! Let us help you get started with the best values in
            the country on new coverage. You can SAVE hundreds or even thousands
            of dollars by requesting a FREE quote from Lifequote Savings. Our
            service will take you less than 5 minutes to complete. Shop and
            compare. SAVE up to 70% on all types of Life insurance!
             HYPERLINK Click Here For Your
            Free Quote!
          Protecting your family is the best investment you'll ever
          make!
      If you are in receipt of this email
      in error and/or wish to be removed from our list,  HYPERLI